In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My\ Drive/tensorcheck

/content/drive/My Drive/tensorcheck


In [0]:
import pickle as pk
f1 = open("angle.pk","rb")
angleArray = pk.load(f1)
f2 = open("image.pk","rb")
imageArray = pk.load(f2)
f1.close()
f2.close()

In [4]:
import tensorflow as tf
import pandas as pd
from skimage import io
from skimage.transform import resize
import numpy as np
import pickle
from keras.models import load_model

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping  
from keras.models import Model
from tensorflow.python import debug as tf_debug
tf.compat.v1.reset_default_graph()

Using TensorFlow backend.


In [0]:
height = 66
width = 200
channels = 3
tf.compat.v1.reset_default_graph()



inputVal = tf.compat.v1.placeholder(tf.float32, shape = [None ,height,width,channels],name = 'features')
output = tf.compat.v1.placeholder(tf.float32, shape = [None,1],name = 'output')
lr = tf.compat.v1.placeholder(tf.float32, name = 'lr')
infinityVar = tf.compat.v1.placeholder(tf.int32,name="infinity")

filter1 = tf.compat.v1.get_variable('weights1', [5, 5, 3, 24], 
                              initializer=tf.compat.v1.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), 
                              dtype=tf.float32)    
conv1 = tf.nn.conv2d(inputVal, filter1, strides = 2, padding='VALID', name = 'conv1')
conv1 = tf.nn.relu(conv1)

filter2 = tf.compat.v1.get_variable('weights2', [5, 5, 24, 36], 
                              initializer=tf.compat.v1.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), 
                              dtype=tf.float32)    
conv2 = tf.nn.conv2d(conv1, filter2, strides = 2, padding='VALID', name = 'conv2')
conv2 = tf.nn.relu(conv2)

filter3 = tf.compat.v1.get_variable('weights3', [5, 5, 36, 48], 
                              initializer=tf.compat.v1.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), 
                              dtype=tf.float32)    
conv3 = tf.nn.conv2d(conv2, filter3, strides = 2, padding='VALID', name = 'conv3')
conv3 = tf.nn.relu(conv3)

filter4 = tf.compat.v1.get_variable('weights4', [3, 3, 48, 64], 
                              initializer=tf.compat.v1.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), 
                              dtype=tf.float32)    
conv4 = tf.nn.conv2d(conv3, filter4, strides = 1, padding='VALID', name = 'conv4')
conv4 = tf.nn.relu(conv4)

filter5 = tf.compat.v1.get_variable('weights5', [3, 3, 64, 64], 
                              initializer=tf.compat.v1.truncated_normal_initializer(stddev=5e-2, dtype=tf.float32), 
                              dtype=tf.float32)    
conv5 = tf.nn.conv2d(conv4, filter5, strides = 1, padding='VALID', name = 'conv5')
conv5 = tf.nn.relu(conv5)

conv5_reshaped = tf.reshape(conv5,shape = (-1,conv5.shape[1]*conv5.shape[2]*conv5.shape[3]))

dense1  = tf.compat.v1.layers.dense(conv5_reshaped,20,activation=tf.nn.relu)
dense2  = tf.compat.v1.layers.dense(dense1,15,activation=tf.nn.relu)
dense3  = tf.compat.v1.layers.dense(dense2,10,activation=tf.nn.relu)
dense4  = tf.compat.v1.layers.dense(dense3,1,activation=tf.nn.relu)

#res = tf.cond(tf.equal(infinityVar,5), lambda: tf.add(5, 1), lambda: tf.add(6, 10))

residual_error = tf.cond(tf.equal(infinityVar,3), lambda:tf.pow(dense4-output,2)/0, lambda:tf.pow(dense4-output,2))
'''
if(infinityVar == 5):
  dense4 = dense4/0
  residual_error = tf.pow(dense4-output,2)/0
  raise ValueError("Infinity exception raised Vipin" + str(residual_error)) 
else:
'''
#residual_error = tf.pow(dense4-output,2)

cost = tf.reduce_mean(residual_error) 
loss_summary = tf.compat.v1.summary.scalar(name="loss",tensor=cost)
r_error = tf.compat.v1.summary.scalar(name="error",tensor=residual_error[0][0])

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(cost)

init = tf.compat.v1.global_variables_initializer()


In [28]:
angleArray.shape

(10, 1)

In [0]:
with tf.compat.v1.Session() as sess:
    sess = tf_debug.TensorBoardDebugWrapperSession(sess, "aca6538b0d55:7003")
    summ_writer = tf.compat.v1.summary.FileWriter("out",sess.graph)
    init.run()
   
    
    epoch = 10
    previous = 0
    for i in range(epoch):
        '''
        r,ls, z ,loss,error,f1,f2,f3,f4,f5,d4,d3,d2,d1 = sess.run((r_error,loss_summary, train_op,cost, residual_error, filter1, filter2,filter3,filter4,filter5, dense4,dense3,dense2,dense1), 
                                feed_dict={inputVal:imageArray, 
                                      output:angleArray,
                                      lr:0.001})
        '''
        r, i, d4,d3,ls,_,loss = sess.run((residual_error,infinityVar, dense4,dense3, loss_summary, train_op, cost), 
                                feed_dict={inputVal:imageArray, 
                                            output:angleArray,
                                            lr:0.001,
                                            infinityVar:i})
        print("***************************************************",loss,i, d4,r) 
        summ_writer.add_summary(ls, i)
        #summ_writer.add_summary(r, i)
        #if(previous != loss):
            #print("***************************************************",loss)    
        #    previous = loss
        #print("error:",error)


75 ops no flops stats due to incomplete shapes.


*************************************************** 0.00030101533 0 [[0.0056981 ]
 [0.0046447 ]
 [0.0044414 ]
 [0.00406766]
 [0.00371568]
 [0.00353077]
 [0.00345256]
 [0.00324651]
 [0.00299477]
 [0.00314516]] [[2.17078250e-05]
 [1.33182334e-06]
 [9.03908642e-07]
 [1.36498898e-06]
 [6.33948594e-06]
 [1.15921765e-04]
 [4.15543385e-04]
 [8.94892029e-04]
 [8.52948113e-04]
 [6.99199911e-04]]


In [0]:
%load_ext tensorboard

In [1]:
%tensorboard --logdir "out" --port 4321 --debugger_port 7003

UsageError: Line magic function `%tensorboard` not found.
